# Model - YOLOv8
# type - One vs All 
# Dataset - extended (class 1 to 5)
# Test - VisDrone (classwise)

# 1. Install Dependencies


In [1]:
!git clone https://github.com/mohitsharma-iitj/SEblock_yolov8.git
%cd SEblock_yolov8
!pip install -r requirements.txt


Cloning into 'SEblock_yolov8'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 205 (delta 36), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (205/205), 3.09 MiB | 14.13 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/kaggle/working/SEblock_yolov8
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.8/530.8 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

# 2. Create dataset function (Annotation)
as the format of annotation is as - object-class,x_center,y_center,width,height (all normalized as this is input format for YOLO5 prediction)

given annotation (in pixel) - bbox_left, bbox_top, bbox_width, bbox_height, score, object_category, truncation, occlusion

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image



os.makedirs('/kaggle/working/yolo_annotations', exist_ok=True)
os.makedirs( '/kaggle/working/yolo_dataset', exist_ok=True)

# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

# Function to convert annotations
def convert_to_yolo_format(filename, annotations, img_width, img_height, given_ ,recieved_path):
    output_annotations_path = recieved_path
    with open(os.path.join(output_annotations_path, filename), 'w') as out_file:
        for annotation in annotations:
            if len(annotation) != 8:
                print(f"Error parsing annotation in {filename}: {annotation}")
                continue
            x_min, y_min, width, height, score, class_id, truncation, occlusion = map(float, annotation)
            if(class_id==0):
                continue
            # Convert to YOLO format
            class_id = class_id -1
            if(class_id == given_):
                class_id = 0
                center_x = x_min + (width / 2.0)
                center_y = y_min + (height / 2.0)
                center_x /= img_width
                center_y /= img_height
                width /= img_width
                height /= img_height

                out_file.write(f"{int(class_id)} {center_x} {center_y} {width} {height}\n")


In [3]:
def create_files_train_val(given_):
    # Process all annotation files    # Paths to the dataset
    annotations_path = '/kaggle/input/extended-dataset/Expanded_dataset/annotations'
    images_path = '/kaggle/input/extended-dataset/Expanded_dataset/images'
    output_annotations_path = '/kaggle/working/yolo_annotations'
    
    shutil.rmtree('/kaggle/working/yolo_dataset')
    shutil.rmtree('/kaggle/working/yolo_annotations')

    os.makedirs('/kaggle/working/yolo_annotations', exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/images','train'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/images','val'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/labels','train'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/labels','val'), exist_ok=True)
    
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)


    # Split the data
    image_files = os.listdir(images_path)
    train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

    # Move files
    for file in train_files:
        shutil.copy(os.path.join(images_path, file), '/kaggle/working/yolo_dataset/images/train/')
        shutil.copy(os.path.join(output_annotations_path, file.replace('.jpg', '.txt')), '/kaggle/working/yolo_dataset/labels/train/')

    for file in val_files:
        shutil.copy(os.path.join(images_path, file), '/kaggle/working/yolo_dataset/images/val/')
        shutil.copy(os.path.join(output_annotations_path, file.replace('.jpg', '.txt')), '/kaggle/working/yolo_dataset/labels/val/')


## for test from visdrone

In [4]:
def create_files_test(given_):
    # Paths to the dataset
    annotations_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/annotations'
    images_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/images'
        
    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'labels/test')
    output_images_path = os.path.join(output_dataset_path, 'images/test')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)

    
    # Process all annotation files
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

In [5]:
def classes_label(class_name):
    create_files_train_val(class_name)
    create_files_test(class_name)


# 3. Import yolov8

In [6]:
import wandb
from ultralytics import YOLO


# 4. Train Class wise



# Class 0 - pedestrian

In [7]:
classes_label(0)

In [8]:
data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['pedestrian']  # class names
"""

with open('/kaggle/working/yolo_dataset/data0.yaml', 'w') as file:
    file.write(data_config)

    


In [9]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")

model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data0.yaml', epochs=25, imgsz=640, batch=16, name='pedestrian_1')



WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295

In [10]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data0.yaml', split='val')


Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:08<00:00,  1.99it/s]
                   all        261       2457      0.781      0.517      0.592      0.341
Speed: 0.2ms preprocess, 12.3ms inference, 0.0ms loss, 3.5ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/pedestrian_1/predictions.json...
Results saved to /kaggle/working/SEblock_yolov8/runs/d

In [11]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/pedestrian_1/weights/best.pt')

# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data0.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 413 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1134.84it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   2%|▏         | 2/101 [00:03<02:59,  1.81s/it]WARNING ⚠️ NMS time limit 1.300s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:29<00:00,  3.42it/s]
                   all       1610      21006    

# Class 1 - People

In [12]:
classes_label(1)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['people']  # class names
"""

with open('/kaggle/working/yolo_dataset/data1.yaml', 'w') as file:
    file.write(data_config)

In [13]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")


model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data1.yaml', epochs=25, imgsz=640, batch=16, name='people_1')


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [14]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data1.yaml', split='val')




Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 146 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:46<00:00,  2.73s/it]
                   all        261        810      0.596      0.434      0.425      0.175
Speed: 0.2ms preprocess, 3.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/people_1/predictions.json...
Results saved to /kaggle/working/SEblock_yolov8/runs/detec

In [15]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/people_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data1.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 813 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1365.29it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:20<00:00,  4.85it/s]
                   all       1610       6376    0.00726     0.0022   0.000855   0.000209
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/val/pr

# class 2 - bicycle

In [16]:
classes_label(2)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['bicycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data2.yaml', 'w') as file:
    file.write(data_config)


In [17]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
 
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data2.yaml', epochs=25, imgsz=640, batch=16, name='bicycle_1')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [18]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data2.yaml', split='val')



Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 216 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [01:26<00:00,  5.09s/it]
                   all        261        124   1.28e-05    0.00806   6.47e-06   6.47e-07
Speed: 0.3ms preprocess, 3.4ms inference, 0.0ms loss, 4.6ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/bicycle_1/predictions.json...
Results saved to /kaggle/working/SEblock_yolov8/runs/dete

In [19]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/bicycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data2.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 1233 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1529.38it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:22<00:00,  4.54it/s]
                   all       1610       1302   2.07e-06   0.000768   1.04e-06   1.04e-07
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 3.3ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/val/p

# class 3 - car

In [20]:
classes_label(3)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['car']  # class names
"""

with open('/kaggle/working/yolo_dataset/data3.yaml', 'w') as file:
    file.write(data_config)





In [21]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    

model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data3.yaml', epochs=25, imgsz=640, batch=16, name='car_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [22]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data3.yaml', split='val')



Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 11 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [01:30<00:00,  5.33s/it]
                   all        261       3396   0.000128    0.00294   6.41e-05   1.86e-05
Speed: 0.2ms preprocess, 3.4ms inference, 0.0ms loss, 3.1ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/car_1/predictions.json...
Results saved to /kaggle/working/SEblock_yolov8/runs/detect/ca

In [23]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/car_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data3.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 80 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1049.66it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:27<00:00,  3.68it/s]
                   all       1610      28074   0.000135    0.00232   6.75e-05   1.81e-05
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 2.6ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/val/pre

# Class 4 - van

In [24]:
classes_label(4)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['van']  # class names
"""

with open('/kaggle/working/yolo_dataset/data4.yaml', 'w') as file:
    file.write(data_config)



In [25]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    

model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data4.yaml', epochs=25, imgsz=640, batch=16, name='van_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [26]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data4.yaml', split='val')



Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 79 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [01:28<00:00,  5.22s/it]
                   all        261        707   2.55e-05    0.00283   1.28e-05    3.2e-06
Speed: 0.2ms preprocess, 3.3ms inference, 0.0ms loss, 3.3ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/van_1/predictions.json...
Results saved to /kaggle/working/SEblock_yolov8/runs/detect/va

In [27]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/van_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data4.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 442 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1219.87it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:23<00:00,  4.33it/s]
                   all       1610       5771   2.28e-05    0.00191   1.14e-05   4.79e-06
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 2.8ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/val/pr

# Class 5 - truck

In [28]:
classes_label(5)

data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['truck']  # class names
"""

with open('/kaggle/working/yolo_dataset/data5.yaml', 'w') as file:
    file.write(data_config)


In [29]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data5.yaml', epochs=25, imgsz=640, batch=16, name='truck_1')


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [30]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data5.yaml', split='val')


Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 158 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [01:28<00:00,  5.23s/it]
                   all        261        275   6.39e-05     0.0182   3.29e-05   9.81e-06
Speed: 0.2ms preprocess, 3.4ms inference, 0.0ms loss, 4.3ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/truck_1/predictions.json...
Results saved to /kaggle/working/SEblock_yolov8/runs/detect

In [31]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/truck_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data5.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 860 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1255.40it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:23<00:00,  4.30it/s]
                   all       1610       2659   0.000149     0.0271   7.75e-05    2.5e-05
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 3.0ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/val/pr